In [38]:
import numpy as np

# Constants
L_v = 2.5e6        # Latent heat of vaporization (J/kg)
R_v = 461.5        # Specific gas constant for water vapor (J/(kg·K))
R_air = 287        # Specific gas constant for dry air (J/(kg·K))
T_0 = 273.15       # Reference temperature (K)
e_0 = 611.2        # Reference vapor pressure at T_0 (Pa)
P_atm = 101325     # Atmospheric pressure (Pa)
D = 2.5e-5         # Diffusion coefficient for water vapor in air (m²/s)
lambda_layer = 0.005  # Boundary layer thickness (meters)

# Chamber dimensions
L = 3              # Side length (meters)
H = 9              # Height (meters)

# Wall temperatures (C)
T_b = 20           # Bottom boundary temperature
T_t = 10           # Top boundary temperature
T_s = 20           # Temperature for two sidewalls
T_s_prime = 10     # Temperature for two other sidewalls


# function for calculating saturation vapor pressure
def saturation_vapor_pressure(T_celsius):
    T_kelvin = T_celsius + 273.15
    return e_0 * np.exp((L_v / R_v) * (1 / T_0 - 1 / T_kelvin))

# Saturation vapor pressures
e_b = saturation_vapor_pressure(T_b)
e_t = saturation_vapor_pressure(T_t)
e_s = saturation_vapor_pressure(T_s)
e_s_prime = saturation_vapor_pressure(T_s_prime)

# Mean temperature
T_mean = (T_b + T_t + 2 * (H / L) * T_s + 2 * (H / L) * T_s_prime) / (2 + 4 * (H / L))
T_mean_K = T_mean + 273.15

# Mean vapor pressure
e_mean = (e_b + e_t + 2 * (H / L) * e_s + 2 * (H / L) * e_s_prime) / (2 + 4 * (H / L))

# Air density
rho_a = P_atm / (R_air * T_mean_K)

# Supersaturation
e_s_mean = saturation_vapor_pressure(T_mean)
supersaturation = (e_mean / e_s_mean) - 1

# Wall areas
A_b = L * L
A_t = L * L
A_s = H * L

# Compute q_v 
numerator = (
     e_b / (rho_a * R_v * (T_b + 273.15)) * A_b
    +  e_t / (rho_a * R_v * (T_t + 273.15)) * A_t
    + 2 *  e_s / (rho_a * R_v * (T_s + 273.15)) * A_s
    + 2 *  e_s_prime / (rho_a * R_v * (T_s_prime + 273.15)) * A_s
)

denominator = (
     A_b + A_t + 4 * A_s
)

q_v = numerator / denominator

# Function to calculate flux and mass flow rate
def calculate_flux_and_mass_flow(q_v, e_sat, T_wall, A):
    T_wall_K = T_wall + 273.15
    qv_sat = e_sat / (rho_a * R_v * T_wall_K)
    J = (rho_a * D / lambda_layer) * (qv_sat - q_v)  # Flux (kg/m^2*s)
    mdot = J * A  
    return J, mdot, qv_sat

# Calculate fluxes and mass flow rates for each wall
J_b, mdot_b, qv_sat_b = calculate_flux_and_mass_flow(q_v, e_b, T_b, A_b)
J_t, mdot_t, qv_sat_t = calculate_flux_and_mass_flow(q_v, e_t, T_t, A_t)

# Two Side Walls at T_s
J_s, mdot_s, qv_sat_s = calculate_flux_and_mass_flow(q_v, e_s, T_s, A_s)
mdot_s_total = 2 * mdot_s

# Two Side Walls at T_s_prime
J_s_prime, mdot_s_prime, qv_sat_s_prime = calculate_flux_and_mass_flow(q_v, e_s_prime, T_s_prime, A_s)
mdot_s_prime_total = 2 * mdot_s_prime

# Net mass flow rate
Mdot_net = mdot_b + mdot_t + mdot_s_total + mdot_s_prime_total

# Calculate evaporated mass flow rate
net_evaporation_rate = mdot_b + mdot_s_total + mdot_s_prime_total  




print(f"Mean vapor pressure: {e_mean:.2f} Pa")
print(f"Mean temperature: {T_mean:.2f} °C")
print(f"Mixing ratio (q_v): {q_v:.8f} kg/kg")
print(f"Saturation vapor pressure at mean temperature: {e_s_mean:.2f} Pa")
print(f"Supersaturation: {supersaturation:.4f}")
print(f"Bottom Wall: J = {J_b:.8f} kg/m^2/s, mdot = {mdot_b:.8f} kg/s")
print(f"Top Wall: J = {J_t:.8f} kg/m^2/s, mdot = {mdot_t:.8f} kg/s")
print(f"Two Side Walls (T_s): J = {J_s:.8f} kg/m²/s, mdot_total = {mdot_s_total:.8f} kg/s")
print(f"Two Side Walls (T_s_prime): J = {J_s_prime:.8f} kg/m^2/s, mdot_total = {mdot_s_prime_total:.8f} kg/s")
print(f"Net mass flow rate: {Mdot_net:.8f} kg/s")

mdot_single = mdot_s_total / 2
print(f"Mass flow rate for one evaporating side wall at T_s: {mdot_single:.8f} kg/s")


Mean vapor pressure: 1798.06 Pa
Mean temperature: 15.00 °C
Mixing ratio (q_v): 0.01097859 kg/kg
Saturation vapor pressure at mean temperature: 1716.09 Pa
Supersaturation: 0.0478
Bottom Wall: J = 0.00002014 kg/m^2/s, mdot = 0.00018128 kg/s
Top Wall: J = -0.00002014 kg/m^2/s, mdot = -0.00018128 kg/s
Two Side Walls (T_s): J = 0.00002014 kg/m²/s, mdot_total = 0.00108769 kg/s
Two Side Walls (T_s_prime): J = -0.00002014 kg/m^2/s, mdot_total = -0.00108769 kg/s
Net mass flow rate: 0.00000000 kg/s
Mass flow rate for one evaporating side wall at T_s: 0.00054385 kg/s


In [39]:
delta_0 = 180e-6  # Film thickness in meters
v_film = 0.06  # Falling velocity in m/s
rho_water = 1000  # Density of water in kg/m³

m_dot_film = rho_water * v_film * delta_0

# Compute evaporation distancee
L_evaporation = m_dot_film / J_s 

print(f"Evaporation distance for the falling film: {L_evaporation:.2f} meters")



Evaporation distance for the falling film: 536.18 meters
